In [79]:
# see https://learn.arcgis.com/en/projects/update-real-time-data-with-python/
# Update real-time data with Python API - Map the most recent data

import arcgis

import os
import tempfile

from urllib import request

In [80]:
url = 'https://coralreefwatch.noaa.gov/product/vs/vs_polygons.json'
temp_dir = tempfile.mkdtemp()
filename = os.path.join(temp_dir, 'latest_data.json')
response = request.urlretrieve(url, filename)

In [81]:
print(f'File name: {filename}')

File name: C:\Users\rb17\AppData\Local\Temp\ArcGISProTemp5504\tmpsvzw2f7v\latest_data.json


In [82]:
import json
json_file = open(filename)
data_raw = json.load(json_file) # this json file now exists in a temp dir

In [83]:
with open(filename) as json_file:
    data_raw = json.load(json_file)
    # Do something with the 'data_raw' variable
# Do something else outside the 'with' section. The json_file variable is now closed.

## Create layers from the file

In [84]:
# create a folder named Temp
# arcpy.management.CreateFileGDB(r'D:\ArcGIS\Temp', 'Live.gdb')
# arcpy.env.workspace = os.path.join(r'D:\ArcGIS\Temp', 'Live.gdb')
arcpy.management.CreateFileGDB( temp_dir, 'Live.gdb' )

<Result 'C:\\Users\\rb17\\AppData\\Local\\Temp\\ArcGISProTemp5504\\tmpsvzw2f7v\\Live.gdb'>

In [85]:
# Create dictionarie with two elements, as required by the GeoJSON file format. 
# The type element will refer to the geometry type in the data_raw variable (your JSON). 
# The features element will list the features. For now, the list will be empty, as indicated by a pair of brackets.

data_stations = dict(type=data_raw['type'], features=[])
data_areas = dict(type=data_raw['type'], features=[])

In [86]:
# loop through all features in the data_raw variable. 
# determine each feature's geometry type. 
# append features of specific geometry types to the list of features of the appropriate dictionary
for feat in data_raw['features']:
    if feat['geometry']['type'] == 'Point':
        data_stations['features'].append(feat)
    else: # elif feat['geometry']['type'] in ['MultiPolygon', 'Polygon']:        
        data_areas['features'].append(feat)

In [87]:
# check how many features were loaded into the list.
len(data_stations['features'])

214

In [88]:
data_areas['features'][9]['properties']['name'] # tenth feature

'Beagle Gulf'

In [89]:
data_areas['features'][9]['geometry']['coordinates']

[[[[129.15, -14.3], [129.15, -14.325], [129.15, -14.35], [129.15, -14.375], [129.15, -14.4], [129.15, -14.425], [129.15, -14.45], [129.175, -14.45], [129.2, -14.45], [129.2, -14.475], [129.2, -14.5], [129.225, -14.5], [129.25, -14.5], [129.25, -14.525], [129.25, -14.55], [129.275, -14.55], [129.3, -14.55], [129.3, -14.575], [129.3, -14.6], [129.325, -14.6], [129.35, -14.6], [129.35, -14.625], [129.35, -14.65], [129.375, -14.65], [129.4, -14.65], [129.4, -14.625], [129.4, -14.6], [129.425, -14.6], [129.45, -14.6], [129.45, -14.575], [129.45, -14.55], [129.475, -14.55], [129.5, -14.55], [129.5, -14.525], [129.5, -14.5], [129.525, -14.5], [129.55, -14.5], [129.55, -14.475], [129.55, -14.45], [129.575, -14.45], [129.6, -14.45], [129.6, -14.425], [129.6, -14.4], [129.6, -14.375], [129.6, -14.35], [129.6, -14.325], [129.6, -14.3], [129.575, -14.3], [129.55, -14.3], [129.55, -14.275], [129.55, -14.25], [129.575, -14.25], [129.6, -14.25], [129.6, -14.225], [129.6, -14.2], [129.625, -14.2], [12

In [90]:
# save the dictionaries as JSON files in the temporary directory with the original JSON (temp_dir).
# run the os.path.join() function to create file paths for each new JSON. 
# create with statements and use the json.dump() function to save the dictionaries as JSON files
# Filenames of temp json files 
stations_json_path = os.path.join(temp_dir, 'points.json')
areas_json_path = os.path.join(temp_dir, 'polygons.json')
# Save dictionaries into json files
with open(stations_json_path, 'w') as point_json_file:
    json.dump(data_stations, point_json_file, indent=4)
with open(areas_json_path, 'w') as poly_json_file:
    json.dump(data_areas, poly_json_file, indent=4)

In [91]:
print(stations_json_path)

C:\Users\rb17\AppData\Local\Temp\ArcGISProTemp5504\tmpsvzw2f7v\points.json


In [92]:
print(areas_json_path)

C:\Users\rb17\AppData\Local\Temp\ArcGISProTemp5504\tmpsvzw2f7v\polygons.json


In [93]:
# Convert JSON files to features
arcpy.conversion.JSONToFeatures(stations_json_path, 'alert_stations') 
arcpy.conversion.JSONToFeatures(areas_json_path, 'alert_areas')

<Result 'C:\\Users\\rb17\\AppData\\Local\\Temp\\ArcGISProTemp5504\\tmpzyn67glq\\Live.gdb\\alert_areas'>

## Change the symbology

In [94]:
# Add alert_level field
arcpy.management.AddField('alert_stations', 'alert_level', 'SHORT', field_alias='Alert Level')
arcpy.management.AddField('alert_areas', 'alert_level', 'SHORT', field_alias='Alert Level')

<Result 'alert_areas'>

In [95]:
# Calculate alert_level field
arcpy.management.CalculateField('alert_stations', 'alert_level', "int(!alert!)")
arcpy.management.CalculateField('alert_areas', 'alert_level', "int(!alert!)")

<Result 'alert_areas'>

In [97]:
# Symbology
p = arcpy.mp.ArcGISProject("CURRENT")
m = p.listMaps('Map')[0]

# Points
points_lyr = m.listLayers('alert_*')[0]
points_sym = points_lyr.symbology

In [98]:
# Always change to the GraduatedSymbolsRenderer from the SimpleRenderer
if points_sym.renderer.type != 'SimpleRenderer':
    points_sym.updateRenderer('SimpleRenderer')
points_sym.updateRenderer('GraduatedSymbolsRenderer')
points_sym.renderer.classificationField = 'alert_level'
points_sym.renderer.breakCount = 4

points_labels = ['0 - 1', '> 1 - 2', '> 2 - 3', '> 3 - 4']
points_upperBounds = [1, 2, 3, 4]
points_sizes = [6, 16.50, 27, 37.50] 
layers_colors = [{'RGB': [5, 113, 176, 40]}, {'RGB': [146, 197, 222, 50]},
                   {'RGB': [244, 165, 130, 50]}, {'RGB': [202, 0, 32, 30]}]

In [99]:
# create a for loop to apply the sizes and colors you specified to each class. 
# Your classes are contained in a numeric list from 1 to 4. You'll use i (for integer) as a 
# placeholder variable to represent the numbered classes.

for i in range(4):
    item = points_sym.renderer.classBreaks[i]
    item.symbol.applySymbolFromGallery('Circle', 1)
    item.label = points_labels[i]
    item.upperBound = points_upperBounds[i]
    item.symbol.size = points_sizes[i]
    item.symbol.color = layers_colors[i]

In [100]:
# Update the symbology
points_lyr.symbology = points_sym

## Publish the layers

In [25]:
# Confirm that you are signed in to an ArcGIS Online account
# If you are signed in, the name of your account appears in the upper right corner of ArcGIS Pro
# On the ribbon, click the Share tab. In the Share As group, click Web Layer.
# ...

## Create a feed routine

In [26]:
# see stand-along script